## Preparando l'ambiente

#### Verifica GPU

In [1]:
!nvidia-smi

"nvidia-smi" non � riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


#### Gestione directory drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

base_dir = '/content/drive/MyDrive/Tracking_project'
data_yaml_path = os.path.join(base_dir, 'data.yaml')

## Installazione librerie

#### Ultralytics 

In [ ]:
!pip install ultralytics==8.3.19

from IPython.display import clear_output, display, Image
clear_output()

!yolo settings sync=false

import ultralytics
ultralytics.checks()

#### Supervision

In [ ]:
!pip install supervision[assets]==0.24.0

clear_output()

import supervision as sv
print(sv.__version__)

## Yolov8 Training

In [ ]:
%cd {base_dir}

from ultralytics import YOLO

model = YOLO('yolov8s.pt')

results = model.train(
  task='detect',
  data=data_yaml_path, 
  epochs=100, 
  imgsz=800,
  plots=True,
  patience=30,
  name='first_training'
)

results_dir = results.save_dir
print(f"risultati addestramento salvati in: {results_dir}")

#### Matrice di confusione

In [ ]:
display(Image(filename=f'{results_dir}/confusion_matrix.png', width=600))

#### Grafici metriche d'apprendimento

In [ ]:
display(Image(filename=f'{results_dir}/results.png', width=600))

### Batch di rilevazioni

In [ ]:
display(Image(filename=f'{results_dir}/val_batch0_pred.jpg'))

#### Detection test

In [ ]:
%cd {base_dir}

model_path = os.path.join('runs/detect/first_train8/', 'weights', 'best.pt')

source_test_image = os.path.join(base_dir, 'dataset', 'images', 'test')

model = YOLO(model_path)

prediction_results = model.predict(source=source_test_image, conf=0.25, save=True)

### Stampa rilevazioni su frame

In [ ]:
display(Image(filename=f'{base_dir}/result_image_detection.png', width=600))

## Tracking

In [ ]:
%cd {base_dir}

from nt import fspath
import numpy as np
import supervision as sv

model_path = os.join('runs/detect/first_train8/', 'weights', 'best.pt')
model = YOLO(model_path)

input_path = os.path.join(base_dir, "output.mp4")
output_path = os.path.join(base_dir, "risultato_tracking..mp4")

video_info = sv.VideoInfo.from_video_path(input_path)

byte_tracker = sv.ByteTrack(
  track_activatio_threshold=0.25
  lost_track_buffer=video_info.fps * 2,
  frame_rate=video_info.fps
)

box_annotator = sv.BoxAnnotator(thickness = 2)
trace_annotator = sv.TraceAnnotator(thickness = 2, trace_length=30)

label_annotator = sv.LabelAnnotator(
  text_position=sv.Position.TOP_CENTER,
  text_scale=0.5,
  text_thickness=1,
  text_color=sv.Color.WHITE
)

def process_frame(frame: np.ndarray, index: int) -> np.ndarray:
  results = model(frame, verbose = False)[0]
  detections = sv.Detections.from_ultralytics(results)
  detections = byte_tracker.update_with_detections(detections)

  labels = [
    f"ID:{tracker_id} {model.model.names[class_id]}"
    for class_id, tracker_id in zip(detections.class_id, detections.tracker_id)
  ]

  annotated_frame = frame.copy()
  annotated_frame = trace_annotator.annotate(scene = annotated_frame, detections = detections)
  annotated_frame = box_annotator.annotate(scene = annotated_frame, detections = detections)
  annotated_frame = label_annotator.annotate(scene = annotated_frame, detections = detections, labels = labels)

  return annotated_frame

  sv.process_video(
    source_path = input_path,
    target_path = output_path,
    callback = process_frame
  )

  print(f"Video con tracking salvato in: {output_path}")

## Semi-supervised learning

#### Pseudo labelling

In [ ]:
%cd {base_dir}

import os
from ultralytics import YOLO
import supervision as sv

model_path = os.path.join('runs/detect/first_train8/', 'weights', 'best.pt')
model = YOLO(model_path)

folder_path = os.path.join(base_dir, 'dataset', 'pseudo_labels')
os.makedirs(folder_path, exist_ok=True)

CONFIDENCE = 0.5

frame_generator = sv.get_video_frames_generator("video_tagliato_finale.mp4")

for frame_index, frame in enumerate(frame_generator):
  results = model(frame, verbose = False)[0]

  yolo_format_labels = []
  for box in results.boxes:
    if box.conf[0] >= CONFIDENCE:
      class_id = int(box.cls[0])
      xywhn = box.xywhn[0]
      x_center, y_center, width, height = xywhn

      yolo_format_labels.append(f"{class_id} {x_center} {y_center} {width} {height}")
  
  if yolo_format_labels:
    label_path = os.path.join(folder_path, f"frame_{frame_index:06d}.txt")
    with open(label_path, "w") as f:
      f.write("\n".join(yolo_format_labels))

print(f"Pseudo-labels generate con successo")